In [81]:
'''
events - словарь со всеми возможными событиями в игре, ключи - их глобальная нумерация 
'''
events = {
    "01": "посетили столовую",
    "02": "попытались посетить Кондратьева в первый раз",
    "03": "осмотрели свою комнату",
    "04": "пожаловались на Кондратьева",
    "05": "пожелали приятного аппетита CU-200",
    "06": "спросили про сына у CU-200",
    "07": "спросили про выбор порции у CU-200",
    "08": "ответили на грубость CU-200 в первый раз",
    "09": "ответили на грубость CU-200 во второй раз",
    "10": "посетили главное смотровое место станции",
    "11": "попытались посетить Кондратьева во второй раз",
    "12": "поговорили с Кондратьевым"
}


In [82]:
'''
функция action_choice - принимает на вход настоящую сцену, является переключетем между сценами после выбора игроком следующего действия 
'''
def action_choice(scene):
    print("Варианты действий:")
    
    counter = 1
    '''
    цикл проверяет воспроизводилась ли в игре уже сцена, котороя может логически следовать из настоящей, и проставляет нумерацию при выводе у сцен, которые ещё не воспроизводились
    '''
    for key in scene.choices.keys():
        if key in scene.you["choices"]:
            continue
        else:
            scene.choices[key][0] = f"{counter}"
            print(scene.choices[key][0], scene.choices[key][1])
            counter += 1

    replay = True

    '''
    цикл обрабатывает ввод пользователя, если ввод корректен, воспроизводит следующую сцену строкой next_scene.play(), если ввод некорректен, просит повторить попытку ввода 
    '''
    while replay:
        response = input(">") 
        print("\n")
        for key in scene.choices.keys():
            if key in scene.you["choices"]:
                continue
            
            else:
                if scene.choices[key][0] == response:
                    replay = False
                    scene.you["choices"].append(key)
                    next_scene = scene.choices[key][2](scene.you)
                    next_scene.play()
        if replay:
            print("Неправильный ввод, повоторите попытку")
                


In [83]:
'''
функция mini_game воспроизводит мини-игру внутри сюжета, нужно угадать слово "человечность" по буквам, угаданные буквы записывают в множество, чтобы не учитывать повторения 
'''
def mini_game():
    word = "человечность"
    mistakes = 0

    proceed = True
    result = set()
    win = False

    print("Вводите каждый раз одну букву")

    '''
    цикл обрабатывает ввод, проверяет есть ли введённая буква в слове и проверяет корректность
    '''
    while proceed:        
        letter = input("< ")        

        if len(letter) == 1 and letter in word:
            result.add(letter)
            for i in word:
                if i in result:
                    print(i, end = "")

                else:
                    print("_", end = "")
                    
            if len(result) == 9:
                proceed = False
                win = True
                print("\n")
            
        else:
            mistakes += 1
            print("Ошибка ", f"{mistakes}")
            
            if mistakes == 9:
                proceed = False

    return win
    

In [84]:
'''
функция first_ending(scene) - выводит итоги иры в файл и выписывает текст первой концовки
'''
def first_ending(scene):
    with open("results.txt", "w", encoding="utf-8") as file:
        file.write("Итоги игры:\n")
        file.write("В столовой началась потасовка, Вы умерли и не узнали тайну станции:( \n")
        file.write("За это время Вы ")

        for key in scene.you["choices"]:
            file.write(f"{events[key]}")
            file.write(", ")

        file.write("Ваша подозрительность поднялась до ")
        file.write( f"{scene.you["suspicion"]}")

In [85]:
'''
функция secong_ending(scene) - выводит итоги иры в файл и выписывает текст второй концовки
'''
def second_ending(scene):
    with open("results.txt", "w", encoding="utf-8") as file:
        file.write("Итоги игры:\n")
        file.write("Вы не узнали тайну станции, через месяц произошёл взрыв, все погибли:( \n")
        file.write("За это время Вы ")

        for key in scene.you["choices"]:
            file.write(f"{events[key]}")
            file.write(", ")

        file.write("Ваша подозрительность поднялась до ")
        file.write( f"{scene.you["suspicion"]}")

In [86]:
'''
функция third_ending(scene) - выводит итоги иры в файл и выписывает текст третьей концовки
'''
def third_ending(scene):
    with open("results.txt", "w", encoding="utf-8") as file:
        file.write("Итоги игры:\n")
        file.write("Вы разгадали загадку станции и смогли предотвратить её разрушение своей осведомлённостью:) \n")
        file.write("За это время Вы ")

        for key in scene.you["choices"]:
            file.write(f"{events[key]}")
            file.write(", ")

        file.write("Ваша подозрительность поднялась до ")
        file.write( f"{scene.you["suspicion"]}")

In [87]:
'''
класс Introduction соответствует сцене вступления, содержит соответствующий текст и связь с другими сценами  
'''
class Introduction:
    def __init__(self, you):
        self.you = you                        

        '''
        в словаре choices хранятся возможные выборы в данной сцене, ключам соответсвует глобальная для всей игры нумерация выборов, вместо первого нуля внутри списка выбранного ключа функция action_choice записывает номер действия при выводе в консоль   
        '''
        self.choices = {
            "01": [0, "Пойти в столовую (главное место скопления сотрудников станции)", Canteen],
            "02": [0, "Пойти в кабинет лаборанта Кондратьева", Kondratiev_office_1],
            "03": [0, "Осмотреть комнату", Room]
        }

    def play(self):
        print("Между фразами жмите 'enter', выбирайте действие цифрой без скобок и точек\n")
        print("Вы просыпаетесь на космической станции в планетной системе звезды Тау Кита. Ваш взгляд падает на пустой диван, место размещения единственного и любимого сына. По своему уже обыкновению утром Вы обнаруживаете только наполовину сползшую простыню и опрокинутую подушку.")
        input()
        print("'Надеюсь, в этот раз он обойдёт стороной кабинет лаборанта Кондратьева, впрочем-то это и не самое страшное место, где может взбрести в голову поиграть восьмилетнему мальчишке на космической станции' - вздыхаете Вы.\n" )
        input()

        action_choice(self)

In [88]:
'''
класс Room соответствует сцене в комнате, имеет структуру аналогичную классу Introduction 
'''
class Room:
    def __init__(self, you):
        self.you = you
        self.you["suspicion"] += 1

        self.choices = {
            "01": [0, "Пойти в столовую (главное место скопления сотрудников станции)", Canteen],
            "02": [0, "Пойти в кабинет лаборанта Кондратьева", Kondratiev_office_1]
        }

    def play(self):
        print("Вы видите свою кровать неожиданно приличных размеров, рабочий стол с компьютером и всяческими мониторами для комфортной работы, новенького плюшевого мишку. ")
        input()
        print("Вы призадумались об архаике последнего, хотя прекрасно помните, как он здесь оказался. ")
        input()

        action_choice(self)
        
    

In [89]:
'''
класс Kondratiev_office_1 соответствует сцене в кабинете Кондратьева, имеет структуру аналогичную классу Introduction 
'''
class Kondratiev_office_1:
    def __init__(self, you):
        self.you = you
        self.you["suspicion"] += 1

        '''
        условная конструкция проверяет когда именно пользователь попадает в кабинет Кондратьева
        '''
        if "06" in self.you["choices"]:
            self.choices = {
                "11": [0, "Постучать ещё раз", Kondratiev_office_2],
                "10": [0, "Пойти на главное смотровое место станции", Observation_deck]
            }

        else:
            self.choices = {
                "01": [0, "Пойти в столовую (главное место скопления сотрудников станции)", Canteen], 
            }

    def play(self):
        print("Вы подходите к кабинету лаборанта Кондратьева, вежливо стучите в дверь и приговариваете.")
        input()
        print("Вы: Уважаемый Савелий Анатольевич, извините за беспоко...")
        input()
        print("Лаборант Кондратьев: Извините, я очень занят, времени на бездельников, увы, не имею! Все рабочие вопросы направляйте через соответствующее устройство в вашей комнате, в нём уже предусмотрена АВТОМАТИЧЕСКАЯ обработка спама. ")
        input()
        
        action_choice(self)



In [90]:
'''
класс Kondratiev_office_1 соответствует сцене в столовой, имеет структуру аналогичную классу Introduction 
'''
class Canteen:
    def __init__(self, you):
        self.you = you

        self.choices = {
            "04": [0, "Пожаловаться на лаборанта Кондратьева", Complain_about_Kondratiev],
            "05": [0, "Пожелать приятного аппетита", Enjoy_your_meal],
            "06": [0, "Спросить про сына", Ask_about_son]
        }

    def play(self):
        print("Вы входите в просторное помещение с новейшим технологическим устройством, позволяющим синтезировать и хранить энергетические напитки, и небольшой группой сотрудников, желающих подкрепиться. Среди десятка лиц Вы обнаруживаете своего старого товарища CU-200.")
        input()
        print("Он машет вам рукой, на что вы с радостью реагируете и подходите.")
        input()
        print("CU-200: Привет!)  Приятное совпадение, ___ однако.")
        input()
        print("Вы: Да, давненько не виделись. Всё-таки зря я перестал ходить в бассейн, клуб любителей электронной музыки оказался куда менее приятным местом… . Кстати, о более насущных вопросах:")
        input()

        action_choice(self)


In [91]:
'''
класс Complain_about_Kondratiev соответствует сцене с жалобой на Кондратьева, имеет структуру аналогичную классу Introduction 
'''
class Complain_about_Kondratiev:
    def __init__(self, you):
        self.you = you
        self.you["suspicion"] += 1

        self.choices = {
            "05": [0, "Пожелать приятного аппетита", Enjoy_your_meal],
            "06": [0, "Спросить про сына", Ask_about_son]
        }

    def play(self):
        print("Вы: Кондратьев совсем на человека перестаёт походить, из кабинета своего не выходит, на вежливые обращения не реагирует.")
        input()
        print("CU-200: Про человеческие материи только не надо высказываться. ___ Не видишь, что ли, в каких условиях живём?!")
        input()
        print("Вы немного напряглись от такого ответа ")
        input()
        print("Вы: Всё-таки он переходит некоторые границы. Сам помнишь, как мой сын вернулся с ссадинами после их незапланированного знакомства. В любом случае,")
        input()

        action_choice(self)

In [92]:
'''
класс Enjoy_your_meal соответствует сцене с пожеланием приятного аппетита, имеет структуру аналогичную классу Introduction 
'''
class Enjoy_your_meal:
    def __init__(self, you):
        self.you = you

        if self.you["suspicion"] == 0:
            self.choices = {
            "06": [0, "Спросить про сына", Ask_about_son],
            "04": [0, "Пожаловаться на лаборанта Кондратьева", Complain_about_Kondratiev]
        }
    
        else:
            self.choices = {
            "06": [0, "Спросить про сына", Ask_about_son],
            "04": [0, "Пожаловаться на лаборанта Кондратьева", Complain_about_Kondratiev],
            "07": [0, "Спросить про выбор порции", Ask_about_meal]
        }

    def play(self):
        print("Вы: Приятного аппетита!")
        input()
        print("CU-200: Спасибо! В этот раз напиток даже не дурен. ")
        input()
        print("Вы замечаете, что товарищ взял минимальную из возможных порций.")
        input()

        action_choice(self)       

    

In [93]:
'''
класс Ask_about_son соответствует сцене с вопросом про сына, имеет структуру аналогичную классу Introduction 
'''
class Ask_about_son:
    def __init__(self, you):
        self.you = you
    
        if "02" in self.you["choices"]:
            self.choices = {
                "10": [0, "Пойти на главное смотровое место станции", Observation_deck],
                "11": [0, "Пойти в кабинет Кондратьева", Kondratiev_office_2]
            }
    
        else:
            self.choices = {
                "10": [0, "Пойти на главное смотровое место станции", Observation_deck],
                "02": [0, "Пойти в кабинет Кондратьева", Kondratiev_office_1]
            }
    
    def play(self):
        print("Вы: мой сын снова утром сбежал, ты его случайно не видел?")
        input()
        print("CU-200: видел, мы с ним даже немного поиграли. Сказал, что пойдёт пожелает доброе утро Кондратьеву, __ а потом отправится на главное смотровое место станции. ")
        input()
        print("Вы: спасибо за информацию. Ох уж этот лаборант проклятый, говорил же ему не приближаться к моему сыну!")
        input()
        print("Вас сильно насторожили слова товарища")
        input()

        action_choice(self)
       

In [94]:
'''
класс Ask_about_meal соответствует сцене с вопросом про еду, имеет структуру аналогичную классу Introduction 
'''
class Ask_about_meal:
    def __init__(self, you):
        self.you = you
        self.you["suspicion"] += 1

        self.choices = {
            "06": [0, "Спросить про сына", Ask_about_son],
            "04": [0, "Пожаловаться на лаборанта Кондратьева", Complain_about_Kondratiev],
            "08": [0, "Ответить CU-200", First_response_CU]
        }

    def play(self):
        print("Вы: сидишь на диете?")
        input()
        print("CU-200: да, немного  ")
        input()
        print("Вы: Я бы не советовал увлекаться минимальными порциями, их формула рассчитана исключительно для существования человека в остром дефиците ресурсов. ")
        input()
        print("CU-200: ну так сам и ешь___, сколько нужно")
        input()
        print("Вы: Всё-таки это опасно, человеческий организм не способен долго функционировать с таким питанием. ")
        input()
        print("CU-200: Отвали а____, тебя это не касается ")
        input()
        print("Ранее ваш товарищ не проявлял такой агрессивности ")
        input()

        action_choice(self)
        
        

In [95]:
'''
класс First_response_CU соответствует сцене с первым ответом CU, имеет структуру аналогичную классу Introduction 
'''
class First_response_CU:
    def __init__(self, you):
        self.you = you
        self.you["suspicion"] += 1
        
        self.choices = {
            "06": [0, "Сменить тему, спросить про сына", Ask_about_son],
            "09": [0, "Ответить CU-200", Second_response_CU]
        }

    def play(self):
        print("Вы: тебя что Кондратьев покусал?")
        input()
        print("CU-200: если ты сейчас не остановишься, ___ я дам тебе в морду")
        input()

        action_choice(self)
    

In [96]:
'''
класс Second_response_CU соответствует сцене со вторым ответом CU, имеет структуру аналогичную классу Introduction 
'''
class Second_response_CU:
    def __init__(self, you):
        self.you = you
        self.you["suspicion"] += 1
        
    def play(self):
        print("Вы говорите на эмоциях")
        input()
        print("Вы: Так ты тоже начал забывать своё человеческое начало! ")
        input()
        print("CU-200 бьёт Вас кулаком в лицо, на следующее утро Вы не просыпаетесь.")
        input()
        print("КОНЕЦ ИГРЫ!")

        first_ending(self)


In [97]:
'''
класс Observation_deck соответствует сцене со смотровым место станции, имеет структуру аналогичную классу Introduction 
'''
class Observation_deck:
    def __init__(self, you):
        self.you = you

    def play(self):
        print("Ваш сын, действительно, здесь. Как только Вы входите, он бросается с места и бежит обниматься. Всё обходится воспитательной беседой. Вы счастливы, что ребёнок в целости и сохранности. ")
        input()

        if self.you["suspicion"] < 4:
            print("КОНЕЦ ИГРЫ")
            second_ending(self)

        else:
            print("Спустя некоторое время Вы замечаете, что защитный костюм сына не имеет стандартной застёжки, без которой его невозможно физически ни надеть, ни снять. Что-то здесь неладное думаете Вы, но ничего не предпринимаете.")
            input()
            print("КОНЕЦ ИГРЫ")
            second_ending(self)
                

In [98]:
'''
класс Kondratiev_office_2 соответствует сцене со вторым визитом к Кондратьеву, имеет структуру аналогичную классу Introduction 
'''
class Kondratiev_office_2:
    def __init__(self, you):
        self.you = you

        if self.you["suspicion"] < 3:
            self.choices = {
                "10": [0, "Пойти на смотровую площадку", Observation_deck] 
            }
        else:
            self.choices = {
                "10": [0, "Пойти на смотровую площадку", Observation_deck],
                "12": [0, "Поинтересоваться", Conversation_Kandratiev]
            }

    def play(self):
        print("Вы стоите напротив кабинета в напряжении и начинаете стучать с убедительной силой. ")
        input()
        print("Кондратьев: Раз уж Ваше желание отвлекать меня от разработки, важного в рамках существования ВСЕЙ космической станции, проекта зондирования планеты Тау Киты е настолько неугомонно, я, так уж и быть, выделю минуту своего времени. Чем могу быть полезен?")
        input()
        print("Вы: Я всё прекрасно понимаю, но Ваши контакты с моим ребёнком вызывают у меня вполне оправданное чувство тревоги. Постарайтесь впредь избегать с ним любых пересечений! ")
        input()

        if self.you["suspicion"] < 3:
            print("Вам больше не о чем разговаривать с Кондратьевым.")
            input()
            action_choice(self)

        else:
            print("Вы замечаете некоторое смятение в лице Кондратьева")
            input()

            action_choice(self)
            

In [99]:
'''
класс Conversation_Kandratiev соответствует сцене с разговором Кондратьевым, имеет структуру аналогичную классу Introduction 
'''
class Conversation_Kandratiev:
    def __init__(self, you):
        self.you = you

        self.choice = {
            "10": [0, "Пойти на смотровую площадку", Observation_deck]
        }

    def play(self):
        print("Вы: Ваше выражение лица отнюдь не делает ситуацию более приятной.")
        input()
        print("Кондратьев: Вы правы… . В своё оправдание скажу, что причины моего специфического поведения куда более серьёзные.")
        input()
        print("Вы: Может, Вы тогда поделитесь ими, раз уж это касается моего ребёнка? ")
        input()
        print("Кондратьев: Не сочтите меня за безумца, но разрешите сначала некоторый ребус. Помните старую игру, в которой нужно по буквам разгадывать слово?")
        input()
        print("Вы: Припоминаю, но какое это имеет отношение к делу?")
        input()
        print("Кондратьев: Очень прямое, главное, не волнуйтесь, для Вас этот ребус не составит никакого труда. Постарайтесь придумать слово, которое описывает недостающую вещь на нашей космической станции. Помните, Вы можете совершить целых 9 ошибок, что превращает задачу в сущий пустяк.")
        input()

        if mini_game():
            print("Кондратьев: несмотря на свою простоту, этот тест не раз доказывал свою эффективность, поэтому я готов поделиться с Вами некоторой информацией.")
            input()
            print("Вы: Я весь внимание!")
            input()
            print("Кондратьев: Хорошо, наверное, Вы уже успели заметить подозрительное поведение некоторых сотрудников при упоминании человеческой сущности?")
            input()
            print("Вы: Это имеет место, CU-200 странно себя сегодня вёл …")
            input()
            print("Кондратьев: Так вот, это неспроста… Ваш сын, как и CU-200, как бы правильнее сказать, не являются людьми. Это, можно считать, инопланетные существа, мимикрировавшие под нас. ")
            input()
            print("Вы: Но как такое возможно! Я прекрасно помню день, когда родился мой сын, как я его учил ходить, наши бесконечные игры и космические путешествия. ")
            input()
            print("Кондратьев: Воспоминания иногда оказываются обманчивыми, с некоторой сноровкой можно приблизиться к их пониманию, но даже у меня бывают сложности. Собственно, для этого я и произвёл некоторую проверку. Сущности очень злятся, когда кто-то начинает сомневаться в их человечности, для своей защиты они становятся невменяемыми и опасными.   ")
            input()
            print("Вы: что Вы собираетесь дальше делать с этой информацией?")
            input()
            print("Кондратьев: пройдёмте, я покажу Вам свои некоторые наработки!")
            input()
            print("КОНЕЦ ИГРЫ")
            
            third_ending(self)

        else:
            print("Кондратьев: Извините, я не готов разговаривать с Вами о волнующих меня вещах. ")
            input()

            action_choice(self)
                    


In [100]:
'''
you содержит уровень подозрительности suspicion, который влияет на сюжет, и хранит уже пройденные сцены  
'''
you = dict(suspicion = 0, choices = []) 
start = Introduction(you)
start.play()

Между фразами жмите 'enter', выбирайте действие цифрой без скобок и точек

Вы просыпаетесь на космической станции в планетной системе звезды Тау Кита. Ваш взгляд падает на пустой диван, место размещения единственного и любимого сына. По своему уже обыкновению утром Вы обнаруживаете только наполовину сползшую простыню и опрокинутую подушку.


'Надеюсь, в этот раз он обойдёт стороной кабинет лаборанта Кондратьева, впрочем-то это и не самое страшное место, где может взбрести в голову поиграть восьмилетнему мальчишке на космической станции' - вздыхаете Вы.



Варианты действий:
1 Пойти в столовую (главное место скопления сотрудников станции)
2 Пойти в кабинет лаборанта Кондратьева
3 Осмотреть комнату


> 1




Вы входите в просторное помещение с новейшим технологическим устройством, позволяющим синтезировать и хранить энергетические напитки, и небольшой группой сотрудников, желающих подкрепиться. Среди десятка лиц Вы обнаруживаете своего старого товарища CU-200.


...

Вам больше не о чем разговаривать с Кондратьевым.


Варианты действий:
1 Пойти на смотровую площадку


> 1




Ваш сын, действительно, здесь. Как только Вы входите, он бросается с места и бежит обниматься. Всё обходится воспитательной беседой. Вы счастливы, что ребёнок в целости и сохранности. 


КОНЕЦ ИГРЫ
